# 03 SDK | 04 Multi Modal Vision

## TODO



### Further information



## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb))


## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.


In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI4Vision_APIKEY") ?? "SKIT_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI4Vision_ENDPOINT") ?? "SKIT_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_AOAI4Vision_DEPLOYMENTNAME") ?? "SKIT_AOAI1106_DEPLOYMENTNAME not found";
string storageConnectionString = Environment.GetEnvironmentVariable("SKIT_STORAGE_CONNECTIONSTRING") ?? "SKIT_STORAGE_CONNECTIONSTRING not found";
string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);


Console.WriteLine($"OpenAI Client created...");

Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.12
    DotNetEnv, 2.5.0

OpenAI Client created...
```

## Step 2: Upload image to Azure Storage and crate Shared Access Signature

We expect the LLM to identify the attraction provided in this image: ![LittleMermaid](../../assets/docs/03_SDK/LittleMermaid.jpg) 

It should deliver the name, location and country of the attraction. The LLM expects a Url to the image which should be analyzed therefore we upload the image to a Storage Account and crate a Shared Access Signature which will be provided to the LLM. 

In [ ]:
#r "nuget: Azure.Storage.Blobs"

using Azure.Storage.Blobs;
using Azure.Storage.Sas;

string containerName = "skit";
string blobName = "LittleMermaid.jpg";
string localFilePath = "../../assets/docs/03_SDK/LittleMermaid.jpg";
string sasUrl = "";

BlobServiceClient blobServiceClient = new BlobServiceClient(storageConnectionString);
BlobContainerClient containerClient = blobServiceClient.GetBlobContainerClient(containerName); 
containerClient.CreateIfNotExists();

BlobClient blobClient = containerClient.GetBlobClient(blobName);
using (FileStream fileStream = new FileStream(localFilePath, FileMode.Open)){
    await blobClient.UploadAsync(fileStream, true);
    BlobSasBuilder sasBuilder = new BlobSasBuilder() {
        BlobContainerName = containerName,
        BlobName = blobClient.Name,
        Resource = "b"
    };
    sasBuilder.ExpiresOn = DateTimeOffset.UtcNow.AddDays(1);
    sasBuilder.SetPermissions(BlobContainerSasPermissions.Read);

    sasUrl = blobClient.GenerateSasUri(sasBuilder).ToString();
}
Console.WriteLine($"Blob uploaded to {sasUrl}");


Expected output: 
```
Blob uploaded to <<Url with SaS>>
```

## Step 2: Compose ChatCompletionsOptions

Each chat would follow similar structure, where _System_, _Agent_ and _User_ messages are added in sequence. Parameters, such as _Temperature_ could be set per call. We can provide the the image url and the user message containing instructions how to process the image using: 

```csharp
new ChatRequestUserMessage(
    new ChatMessageTextContentItem(userMessage),
    new ChatMessageImageContentItem(new Uri(sasUrl))
)
```


In [ ]:
//Define System Prompt
string systemMessage = @" 
    You are an assistant who helps travel agency staff to find interesting attractions around the world. 
    You name the place of interest, the city and the country in which the attraction is located. 
    You do not provide any further information.
"; 

string userMessage = @"
    Which place of interest is this?
";

//Compose Chat (Simplified - No few shot learning in this example)
ChatCompletionsOptions chatCompletionsOptions = new ChatCompletionsOptions()
{
    DeploymentName = chatCompletionDeploymentName,
    Messages =
    {
        new ChatRequestSystemMessage(systemMessage),
        new ChatRequestUserMessage(userMessage),
        new ChatRequestUserMessage(
            new ChatMessageTextContentItem(userMessage),
            new ChatMessageImageContentItem(new Uri(sasUrl))
        )
    },
    MaxTokens = 100, 
    NucleusSamplingFactor = 0.1f,
    Temperature = 0.1f,
};

Console.WriteLine($"ChatCompletionsOptions created...");



Expected output:

```
ChatCompletionsOptions created...
```

## Step 3: Call ChatCompletion API



In [ ]:
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);
Console.WriteLine(response.Value.Choices[0].Message.Content);

Expected output:

```
The Little Mermaid statue, Copenhagen, Denmark.
```

## Next Steps

- The concept of Embeddings allows transforming information into a numerical representation preserving the semantic context of the information: [Demo Embeddings](../04_Embeddings/01_BasicEmbeddings.ipynb)